In [1]:
import torch
from transformers import AutoTokenizer,OlmoeForCausalLM, OlmoeConfig
torch.set_printoptions(profile='simple')
model_path = "shuyuej/OLMoE-1B-7B-0924-GPTQ"#"allenai/OLMoE-1B-7B-0924"
config = OlmoeConfig.from_pretrained(model_path,num_experts_per_tok=4)
model = OlmoeForCausalLM.from_pretrained(model_path,config=config,
                                         device_map="auto",
                                         torch_dtype=torch.float16,
                                         )
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()

🚨 `use_probabilistic_routing` is part of OlmoeModel.forward's signature, but not documented. Make sure to add it to the docstring of the function in X:\Alles Unwichtige\Uni rub\fächer\Master\Semester 4 SS25\Engineering for Large Language Models\transformers\src\transformers\models\olmoe\modeling_olmoe.py.
🚨 `prob_routing_temp` is part of OlmoeModel.forward's signature, but not documented. Make sure to add it to the docstring of the function in X:\Alles Unwichtige\Uni rub\fächer\Master\Semester 4 SS25\Engineering for Large Language Models\transformers\src\transformers\models\olmoe\modeling_olmoe.py.
🚨 `use_probabilistic_routing` is part of OlmoeForCausalLM.forward's signature, but not documented. Make sure to add it to the docstring of the function in X:\Alles Unwichtige\Uni rub\fächer\Master\Semester 4 SS25\Engineering for Large Language Models\transformers\src\transformers\models\olmoe\modeling_olmoe.py.
🚨 `prob_routing_temp` is part of OlmoeForCausalLM.forward's signature, but not do

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
x:\Alles Unwichtige\Uni rub\fächer\Master\Semester 4 SS25\Engineering for Large Language Models\LLM-Eng-Project\.venv\Lib\site-packages\accelerate\utils\modeling.py:808: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  _ = torch.tensor([0], device=i)
INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


INFO  Format: Converting `checkpoint_format` from `gptq` to internal `gptq_v2`.
INFO  Format: Converting GPTQ v1 to v2                                         
INFO  Format: Conversion complete: 0.0802152156829834s                         
INFO  Optimize: `TritonV2QuantLinear` compilation triggered.                   


OlmoeForCausalLM(
  (model): OlmoeModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoeDecoderLayer(
        (self_attn): OlmoeSdpaAttention(
          (q_norm): OlmoeRMSNorm((2048,), eps=1e-05)
          (k_norm): OlmoeRMSNorm((2048,), eps=1e-05)
          (k_proj): TritonV2QuantLinear()
          (o_proj): TritonV2QuantLinear()
          (q_proj): TritonV2QuantLinear()
          (v_proj): TritonV2QuantLinear()
        )
        (mlp): OlmoeSparseMoeBlock(
          (experts): ModuleList(
            (0-63): 64 x OlmoeMLP(
              (act_fn): SiLU()
              (down_proj): TritonV2QuantLinear()
              (gate_proj): TritonV2QuantLinear()
              (up_proj): TritonV2QuantLinear()
            )
          )
          (gate): TritonV2QuantLinear()
        )
        (input_layernorm): OlmoeRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): OlmoeRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): O

In [ ]:
if "inputs" in globals():
    del inputs
inputs = tokenizer("A short fantasy story with children:", return_tensors="pt").to(model.device)

# prob_routing_temp cannot be lower than about 0.0001. 
# 0.00001 causes cuda assert

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=20,
        use_probabilistic_routing=True,
        prob_routing_temp=.0001,
        #output_router_logits=True,
        #return_dict=True
    )

print(tokenizer.decode(outputs[0]))

A short fantasy story with children: and and and and and and and and and and and and and and and and and and and and
